In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

#----------------------function definitions------------------------------------
def init_spins(L):
    # allocate random spin orientations [-1,+1] to LxL lattice
    spins = np.random.choice([-1, 1],size=(L, L))
    return spins

def delta_energy(spins, m, n):
    # calculate interaction energy of a spin at (m,n) with neighboring spins
    L = spins.shape[0]
    J = 1
    spin = spins[m, n]
    neighbors = spins[(m+1)%L, n] + spins[(m-1)%L, n] + spins[m, (n+1)%L] + spins[m, (n-1)%L]
    return 2 * J * spin * neighbors

def metropolis(spins, kT):
    # randomly select spin and calculate the energy associated with the spin
    # use metropolis criterion at temp=kT to decide whether to flip spin
    # repeat for numSwaps=1000*L**2
    L = spins.shape[0]
    numSwaps = 1000*L**2
    for _ in range(numSwaps):
        m, n = np.random.randint(0, L, size=2)
        dE = delta_energy(spins, m, n)
        if dE < 0 or np.random.rand() < np.exp(-dE / kT):
            spins[m, n] *= -1
    return spins

def measure(spins):
    # magnetiztion is the average spin value
    return abs(np.mean(spins))

#-----------------------------main program-------------------------------------
L = 50  
temp = np.array([1.5, 1.8, 2.1, 2.2, 2.27, 2.4, 2.5, 2.7, 3.0, 3.5])
magnetization = []

spins = init_spins(L)

start_time = time.time()

for kT in temp:
    spins = metropolis(spins, kT)
    mag = measure(spins)
    magnetization.append(mag)
    
    plt.imshow(spins,cmap='bwr',interpolation='nearest')
    plt.title(f"kT={kT:,.2f}")
    plt.show()
    
end_time = time.time()
print(f"Simulation time: {int(end_time - start_time)} seconds")


---
# Lab 3 An object-oriented Ising model

**PYU33C01 Computational Simulation I, Computational Physics (Christie)**


---
## What are the objectives of this lab?

* Understand the structure and behavior of the 2D Ising model.
* Refactor procedural code into an object-oriented design.
* Use classes to encapsulate data and behavior.
* Interact with GenAI to debug, extend, and analyze the object-oriented  code.

---
## The procedural implementation of the Ising model

The Python file _IsingModelProcedural.py_ can be downloaded from BlackBoards. It contains the code developed in the lecture notebook to simulate a 2D Ising spin system composed of an $L \times L$  square array of spins. The code uses procedural programming techniques and is based on four function definitions. The skeleton outlines are shown below:

```python
def init_spins(L):
    # allocate random spin orientations [-1,+1] to a 2D NumPy LxL array

def delta_energy(spins, m, n):
    # calculate energy of a spin at (m,n) with its nearest neighborings. 
    # It can be negative or positive depending on the value of the spin at (m,n) and the values of neighboring spins

def metropolis(spins, kT):
    # randomly select spin and calculate the energy associated with the spin
    # use metropolis criterion to decide whether to flip the spin or not at temperature kT
    # repeat for numSwaps=1000*L**2

def measure(spins):
    # magnetiztion is the average spin value
```

---
The main code for running the procedutal simulation is

```python
L = 50  
temp = np.array([1.5, 1.8, 2.1, 2.2, 2.27, 2.4, 2.5, 2.7, 3.0, 3.5])
magnetization = []

spins = init_spins(L)

start_time = time.time()

for kT in temp:
    spins = metropolis(spins, kT)
    mag = measure(spins)
    magnetization.append(mag)
    
    plt.imshow(spins,cmap='bwr',interpolation='nearest')
    plt.title(f"kT={kT:,.2f}")
    plt.show()
    
end_time = time.time()
print(f"Simulation time: {int(end_time - start_time)} seconds")
```
The function _metropolis_ calls _delta_energy_ internally.

---
# Lab 3 An object-oriented Ising model

**PYU33C01 Computational Simulation I, Computational Physics (Christie)**


---
## What are the objectives of this lab?

* Understand the structure and behavior of the 2D Ising model.
* Refactor procedural code into an object-oriented design.
* Use classes to encapsulate data and behavior.
* Interact with GenAI to debug, extend, and analyze the object-oriented  code.

---
## The procedural implementation of the Ising model

The Python file _IsingModelProcedural.py_ can be downloaded from BlackBoards. It contains the code developed in the lecture notebook to simulate a 2D Ising spin system composed of an $L \times L$  square array of spins. The code uses procedural programming techniques and is based on four function definitions. The skeleton outlines are shown below:

```python
def init_spins(L):
    # allocate random spin orientations [-1,+1] to a 2D NumPy LxL array

def delta_energy(spins, m, n):
    # calculate energy of a spin at (m,n) with its nearest neighborings. 
    # It can be negative or positive depending on the value of the spin at (m,n) and the values of neighboring spins

def metropolis(spins, kT):
    # randomly select spin and calculate the energy associated with the spin
    # use metropolis criterion to decide whether to flip the spin or not at temperature kT
    # repeat for numSwaps=1000*L**2

def measure(spins):
    # magnetiztion is the average spin value
```

---
The main code for running the procedutal simulation is

```python
L = 50  
temp = np.array([1.5, 1.8, 2.1, 2.2, 2.27, 2.4, 2.5, 2.7, 3.0, 3.5])
magnetization = []

spins = init_spins(L)

start_time = time.time()

for kT in temp:
    spins = metropolis(spins, kT)
    mag = measure(spins)
    magnetization.append(mag)
    
    plt.imshow(spins,cmap='bwr',interpolation='nearest')
    plt.title(f"kT={kT:,.2f}")
    plt.show()
    
end_time = time.time()
print(f"Simulation time: {int(end_time - start_time)} seconds")
```
The function _metropolis_ calls _delta_energy_ internally.

---
## Task 1

Using your GenAI assistant, generate an object-oriented program to simulate the 2D Ising model. The minimum requirements are that the object-oriented version should define a class for a spin object and the spins should be encapsulated in a IsingLattice class. It should have the same number of spins and run over the same temperature range as the procedural program.

Be aware that uploading the procedural program and asking your GenAI assistant to transcribe it into an object-oriented version tends to produce a program that does not take advantage of many object-oriented features.

---
## Task 2

Test and critically evaluate the code. Make (at least) 3 modifications/improvements to your new Python class.

---
## Task 3

Plot the average magnetisation $\langle M \rangle$ as a function of the magnetic field $h$ for system size $L=10$ and two different temperatures: $k_BT=1.0$ and $k_BT=4.0$ (assuming $J=1$).

---
## Task 4

Plot the average magnetization $\langle M \rangle$ for $h=0$. Find the critical temperature $T_c$ of the phase transition between ferromagnetic and paramagnetic phases.

---
## Task 5 Bonus

Explore the physics of your final object-oriented model. You may wish to consider the eﬀect of:

* system size and shape
* initial state
* number of Metropolis samples
* sweeping through lattice sites sequentially or randomly
* other lattice geometries (e.g. hexagonal, triangular, etc.)
* diﬀerent boundary conditions
* methods to visualise the microstate
* ...

feel free to explore and get creative!

---
## Task 1

Using your GenAI assistant, generate an object-oriented program to simulate the 2D Ising model. The minimum requirements are that the object-oriented version should define a class for a spin object and the spins should be encapsulated in a IsingLattice class. It should have the same number of spins and run over the same temperature range as the procedural program.

Be aware that uploading the procedural program and asking your GenAI assistant to transcribe it into an object-oriented version tends to produce a program that does not take advantage of many object-oriented features.

---
## Task 2

Test and critically evaluate the code. Make (at least) 3 modifications/improvements to your new Python class.

---
## Task 3

Plot the average magnetisation $\langle M \rangle$ as a function of the magnetic field $h$ for system size $L=10$ and two different temperatures: $k_BT=1.0$ and $k_BT=4.0$ (assuming $J=1$).

---
## Task 4

Plot the average magnetization $\langle M \rangle$ for $h=0$. Find the critical temperature $T_c$ of the phase transition between ferromagnetic and paramagnetic phases.

---
## Task 5 Bonus

Explore the physics of your final object-oriented model. You may wish to consider the eﬀect of:

* system size and shape
* initial state
* number of Metropolis samples
* sweeping through lattice sites sequentially or randomly
* other lattice geometries (e.g. hexagonal, triangular, etc.)
* diﬀerent boundary conditions
* methods to visualise the microstate
* ...

feel free to explore and get creative!

Some ideas - 
record how many times each spin site flips, then graph this as a heat map to visualise boudaries?